In [19]:
import pandas as pd

In [20]:
data = pd.read_csv('wiki_movie_plots_deduped.csv')

In [21]:
data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [24]:
data = data.loc[:1000, ['Plot']]

In [25]:
data.shape

(1001, 1)

In [26]:
from nltk.tokenize import sent_tokenize
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ellivalla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
from tqdm import tqdm
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

In [31]:
data['sentences'] = data.Plot.progress_map(sent_tokenize)
data['sentences'].head(1).tolist()[0][:3] # Print the first 3 sentences of the 1st article

['A bartender is working at a saloon, serving drinks to customers.',
 "After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside.",
 'They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head.']

In [32]:
from nltk.tokenize import word_tokenize

In [33]:
data['tokens_sentences'] = data['sentences'].progress_map(lambda sentences: [word_tokenize(sentence) for sentence in sentences])
print(data['tokens_sentences'].head(1).tolist()[0][:3])


[['A', 'bartender', 'is', 'working', 'at', 'a', 'saloon', ',', 'serving', 'drinks', 'to', 'customers', '.'], ['After', 'he', 'fills', 'a', 'stereotypically', 'Irish', 'man', "'s", 'bucket', 'with', 'beer', ',', 'Carrie', 'Nation', 'and', 'her', 'followers', 'burst', 'inside', '.'], ['They', 'assault', 'the', 'Irish', 'man', ',', 'pulling', 'his', 'hat', 'over', 'his', 'eyes', 'and', 'then', 'dumping', 'the', 'beer', 'over', 'his', 'head', '.']]


In [34]:
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ellivalla/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [35]:
data['POS_tokens'] = data['tokens_sentences'].progress_map(lambda tokens_sentences: [pos_tag(tokens) for tokens in tokens_sentences])
print(data['POS_tokens'].head(1).tolist()[0][:3])


[[('A', 'DT'), ('bartender', 'NN'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('a', 'DT'), ('saloon', 'NN'), (',', ','), ('serving', 'VBG'), ('drinks', 'NNS'), ('to', 'TO'), ('customers', 'NNS'), ('.', '.')], [('After', 'IN'), ('he', 'PRP'), ('fills', 'VBZ'), ('a', 'DT'), ('stereotypically', 'RB'), ('Irish', 'JJ'), ('man', 'NN'), ("'s", 'POS'), ('bucket', 'NN'), ('with', 'IN'), ('beer', 'NN'), (',', ','), ('Carrie', 'NNP'), ('Nation', 'NNP'), ('and', 'CC'), ('her', 'PRP$'), ('followers', 'NNS'), ('burst', 'JJ'), ('inside', 'RB'), ('.', '.')], [('They', 'PRP'), ('assault', 'VBP'), ('the', 'DT'), ('Irish', 'NNP'), ('man', 'NN'), (',', ','), ('pulling', 'VBG'), ('his', 'PRP$'), ('hat', 'NN'), ('over', 'IN'), ('his', 'PRP$'), ('eyes', 'NNS'), ('and', 'CC'), ('then', 'RB'), ('dumping', 'VBG'), ('the', 'DT'), ('beer', 'NN'), ('over', 'IN'), ('his', 'PRP$'), ('head', 'NN'), ('.', '.')]]


In [36]:
from nltk.corpus import wordnet
nltk.download('wordnet')

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ellivalla/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
# Lemmatizing each word with its POS tag, in each sentence
data['tokens_sentences_lemmatized'] = data['POS_tokens'].progress_map(
    lambda list_tokens_POS: [
        [
            lemmatizer.lemmatize(el[0], get_wordnet_pos(el[1])) 
            if get_wordnet_pos(el[1]) != '' else el[0] for el in tokens_POS
        ] 
        for tokens_POS in list_tokens_POS
    ]
)

In [38]:
data['tokens_sentences_lemmatized'].head(1).tolist()[0][:3]

[['A',
  'bartender',
  'be',
  'work',
  'at',
  'a',
  'saloon',
  ',',
  'serve',
  'drink',
  'to',
  'customer',
  '.'],
 ['After',
  'he',
  'fill',
  'a',
  'stereotypically',
  'Irish',
  'man',
  "'s",
  'bucket',
  'with',
  'beer',
  ',',
  'Carrie',
  'Nation',
  'and',
  'her',
  'follower',
  'burst',
  'inside',
  '.'],
 ['They',
  'assault',
  'the',
  'Irish',
  'man',
  ',',
  'pull',
  'his',
  'hat',
  'over',
  'his',
  'eye',
  'and',
  'then',
  'dump',
  'the',
  'beer',
  'over',
  'his',
  'head',
  '.']]

In [39]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_verbs = ['say', 'get', 'go', 'know', 'may', 'need', 'like', 'make', 'see', 'want', 'come', 'take', 'use', 'would', 'can']
stopwords_other = ['one', 'mr', 'image', 'de', 'en', 'caption', 'also', 'something']
my_stopwords = stopwords.words('English') + stopwords_verbs + stopwords_other

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellivalla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
from itertools import chain # to flatten list of sentences of tokens into list of tokens

In [41]:
data['tokens'] = data['tokens_sentences_lemmatized'].map(lambda sentences: list(chain.from_iterable(sentences)))
data['tokens'] = data['tokens'].map(lambda tokens: [token.lower() for token in tokens if token.isalpha() 
                                                    and token.lower() not in my_stopwords and len(token)>1])

In [42]:
data['tokens'].head(1).tolist()[0][:30]

['bartender',
 'work',
 'saloon',
 'serve',
 'drink',
 'customer',
 'fill',
 'stereotypically',
 'irish',
 'man',
 'bucket',
 'beer',
 'carrie',
 'nation',
 'follower',
 'burst',
 'inside',
 'assault',
 'irish',
 'man',
 'pull',
 'hat',
 'eye',
 'dump',
 'beer',
 'head',
 'group',
 'begin',
 'wreck',
 'bar']

In [43]:
from gensim.models import Phrases

In [44]:
tokens = data['tokens'].tolist()
bigram_model = Phrases(tokens)
trigram_model = Phrases(bigram_model[tokens], min_count=1)
tokens = list(trigram_model[bigram_model[tokens]])
print(tokens[0:10])

[['bartender', 'work', 'saloon', 'serve', 'drink', 'customer', 'fill', 'stereotypically', 'irish_man', 'bucket', 'beer', 'carrie', 'nation', 'follower', 'burst', 'inside', 'assault', 'irish_man', 'pull', 'hat', 'eye', 'dump', 'beer', 'head_group', 'begin', 'wreck', 'bar', 'smash', 'fixture', 'mirror_break', 'cash', 'register', 'bartender', 'spray_seltzer', 'water', 'nation', 'face', 'group', 'policeman_appear', 'order', 'everybody', 'leave'], ['moon', 'paint', 'smile_face', 'hang', 'park_night', 'young_couple', 'walk_past', 'fence', 'learn', 'railing', 'look', 'moon_smile', 'embrace', 'moon_smile', 'big', 'sit_bench', 'tree', 'moon', 'view', 'block', 'cause', 'frown', 'last_scene', 'man', 'fan', 'woman_hat', 'moon', 'leave', 'sky', 'perch', 'shoulder', 'everything_well'], ['film', 'minute', 'long', 'compose', 'two_shot', 'first', 'girl', 'sit', 'base', 'altar', 'tomb', 'face', 'hidden', 'camera', 'center', 'altar', 'view', 'portal', 'display', 'portrait', 'three', 'lincoln', 'james', '

In [45]:
from gensim import corpora

In [46]:
dictionary_LDA = corpora.Dictionary(tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(tok) for tok in tokens]

In [47]:
from gensim import models
import numpy as np

In [48]:
np.random.seed(123456)
num_topics = 20
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

CPU times: user 4.88 s, sys: 16.6 ms, total: 4.9 s
Wall time: 4.92 s


In [49]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=20):
    print(str(i)+": "+ topic)
    print()

0: 0.016*"girl" + 0.008*"find" + 0.008*"jerry" + 0.007*"marry" + 0.007*"two" + 0.007*"jekyll" + 0.006*"man" + 0.006*"father" + 0.006*"boy" + 0.006*"another" + 0.006*"child" + 0.005*"love" + 0.005*"ann" + 0.005*"meet" + 0.005*"fall_love" + 0.004*"play" + 0.004*"son" + 0.004*"end" + 0.004*"family" + 0.004*"hyde"

1: 0.010*"jack" + 0.008*"father" + 0.007*"find" + 0.006*"child" + 0.006*"kill" + 0.005*"become" + 0.005*"mother" + 0.005*"angela" + 0.005*"dick" + 0.005*"marry" + 0.005*"film" + 0.004*"play" + 0.004*"give" + 0.004*"leave" + 0.004*"home" + 0.004*"return" + 0.004*"rescue" + 0.004*"chick" + 0.004*"karl" + 0.004*"family"

2: 0.016*"harold" + 0.012*"mary" + 0.011*"jim" + 0.008*"however" + 0.006*"leave" + 0.006*"back" + 0.005*"turn" + 0.005*"two" + 0.005*"find" + 0.005*"mother" + 0.005*"time" + 0.004*"rescue" + 0.004*"work" + 0.004*"become" + 0.004*"peggy" + 0.004*"woman" + 0.004*"marry" + 0.004*"order" + 0.004*"grant" + 0.004*"play"

3: 0.010*"tell" + 0.008*"leave" + 0.007*"jean" + 0

In [50]:
lda_model[corpus[0]]

[(15, 0.99427533)]

In [51]:
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary_LDA)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
